# Latest, working great code for Scene Graph Predict

EXPECTED INPUT
```python
one_segment_CLIP_output = {
    "video_stem": str(Path(video_filepath).stem),
    "segment_id": str(Path(video_filepath).stem) + f"_{i}",
    "segment_index": i,
    "total_segments": len(whisper_segments),
    "segment_total_time": whisper_segments[i]['end'] - whisper_segments[i]['start'],
    "captions": whisper_segments[i]['caption'],
    "segment_start_time": whisper_segments[i]['start'],
    "segment_end_time": whisper_segments[i]['end'],
    "num_frames_per_segment": self.num_frames,
    "frame_embeddings": image_feature,
    "text_caption_embeddings": caption_feature,
    "segment_frames": segment_frame,
    "frame_embeddings_shape": image_feature.shape,          # trying the FLATTEN technique!
    "text_caption_embeddings_shape": caption_feature.shape,
    "segment_frames_shape": segment_frame.shape,
    "scene_graph_captions": "comma separated string (list of scene graph outputs)"  # <--- this is all we're adding
}
```

In [1]:
from faster_OpenPSG.predict import Predictor 
my_pred = Predictor()
my_pred.setup()

/home/kastan/utils/miniconda3/envs/openpsg_custom/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


load checkpoint from local path: faster_OpenPSG/checkpoints/epoch_60.pth


In [3]:
# import functiontrace
# functiontrace.trace()
import yappi


import numpy as np
import cv2
from PIL import Image
import time
import glob

# except KeyError as e:
  # print("End of file")
  
yappi.set_clock_type("cpu") # Use set_clock_type("wall") for wall time
yappi.start()

# todo: remove call to show_result()


# glob.glob()
NUM_RELATIONS = 1

path = '/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/parallel_15_clip_output/J-DG5O7gqdY_All4TubeVLOGS_15315_This Is Why We Had To Move To Florida.npz'

start_time = time.monotonic()

# they're always named arr_0 thru num_segments 
np_loaded = np.load(path, allow_pickle=True)
object_list_of_str = []
for segment_index in range(np_loaded['arr_0'].item()['total_segments']):
  # print(np_loaded[f'arr_{segment_index}'].item()['captions'])
  frame = np_loaded[f'arr_{segment_index}'].item()['segment_frames']
  frame = frame.reshape(336, 336, 3)
  frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
  
  # display(Image.fromarray(frame_rgb))
  # print('loaded: ')
  # print(frame_rgb.shape)
  
  curr_sro, result = my_pred.predict(image=frame_rgb, num_rel=NUM_RELATIONS)
  # print("HERE RES")
  # print(result)
  # print(curr_sro)
  
  # for rel in curr_sro:
  #   if 'kissing' in rel:
  #     print(curr_sro)
  #     display(Image.fromarray(frame_rgb))
  
  object_list_of_str.append(", ".join(curr_sro))
  
  # if segment_index > 20:
  #   break
  # break

print(f"⏰ Ran scene graph {len(object_list_of_str), } frames in {(time.monotonic()-start_time):2f} seconds")  
yappi.get_func_stats().print_all()
yappi.get_thread_stats().print_all()

⏰ Ran scene graph (113,) frames in 83.915642 seconds

Clock type: CPU
Ordered by: totaltime, desc

name                                  ncall  tsub      ttot      tavg      
..:3511 ZMQInteractiveShell.run_code  24     0.000220  172.6658  7.194408
.._2492029/4181574521.py:31 <module>  2      0.012526  172.3371  86.16854
..SG/predict.py:61 Predictor.predict  226    0.016055  171.6284  0.759418
..psg/utils/utils.py:284 show_result  226    0.097784  115.5445  0.511259
..inference.py:89 inference_detector  226    0.029259  56.06286  0.248066
..isualizer.py:255 VisImage.__init__  904    0.004609  54.61671  0.060417
..izer.py:266 VisImage._setup_figure  904    0.022301  54.61210  0.060412
..ib/_api/deprecation.py:448 wrapper  1808   0.010310  53.27306  0.029465
..les/module.py:715 PSGTr._call_impl  104..  1.486640  52.79387  0.000506
..v/runner/fp16_utils.py:90 new_func  226    0.002688  52.78421  0.233558
..etectors/base.py:156 PSGTr.forward  226    0.001975  52.78098  0.233544
..ors/base.

In [ ]:
import jsonlines 
import cv2
import json_numpy
from PIL import Image

NUM_RELATIONS = 5
INPUT_FILENAME = '/mnt/storage_ssd/parallel_10_clip_output.jsonl'
# INPUT_FILENAME = '/mnt/storage_ssd/parallel_14_clip_output.jsonl'

with jsonlines.open(INPUT_FILENAME) as reader, jsonlines.open(INPUT_FILENAME, mode='a') as writer:
    sro_list = set()
    for obj_idx, test_sample in enumerate(reader):
        num_frames_counter = 0
        if test_sample['segment_frames'][num_frames_counter] is not None:
            print(test_sample['segment_frames'])
            print(test_sample['segment_frames'][0])
            frame = json_numpy.loads(test_sample['segment_frames'][num_frames_counter])
            frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            # display(Image.display(frame_rgb))
            print('loaded: ')
            print(frame_rgb.shape)
            
            curr_sro = my_pred.predict(image=frame_rgb, num_rel=NUM_RELATIONS)
            sro_list.update(set(curr_sro))
            print(sro_list)
            num_frames_counter += 1
        test_sample['scene_graph_set'] = list(sro_list)
        writer.write(test_sample)
        if obj_idx >= 1:
            break
        break

# Embed with Flan-T5

In [ ]:
# !pip install transformers

In [ ]:
import torch
import accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto", torch_dtype=torch.float16)

In [ ]:
CONTEXT = "{Example: A Two-Bit Gray Code Counter}Let's begin with a two-bit Gray code counter with no inputs.As we mentioned in Notes Set 2.1, a Gray code is a cycle over allbit patterns of a certain length in which consecutive patterns differin exactly one bit.For simplicity, our first few examples are based on counters anduse the internal stateof the FSM as the output values.  You should already knowhow to design combinational logic for the outputs if it were necessary.The inputs to a counter, if any, are typically limited to functionssuch as starting and stopping the counter, controlling the counting direction, and resetting the counter to a particular state.A fully-specified transition diagram for a two-bit Gray code counter appears below.With no inputs, the states simply form a loop, withthe counter moving from one state to the next each cycle.Each state in the diagram is marked with the internal state value S_1S_0 (before the ``/'') and the output Z_1Z_0 (after the ``/''), which are always equal for this counter.Based on the transition diagram, we can fill in the K-maps for the next-state values S_1^+ and S_0^+ as shown to the right of thetransition diagram, then derive algebraic expressions in the usual way to obtainS_1^+=S_0 and S_0^+={{S_1}}.We then use the next-state logic to develop the implementationshown on the far right, completing our first counter design."
PROMPT = "Please answer this person's question accurately, clearly and concicely. Context: "
QUESTION = "Question: What are the inputs and outputs of a Gray code counter? "
input_text = PROMPT + CONTEXT + QUESTION + "Answer: "

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_length=1024)
print(tokenizer.decode(outputs[0]))

In [ ]:
# TODO: read in and make sure the first 5 are perfect. 

with jsonlines.open('/mnt/storage_ssd/JOHAN_parallel_10_clip_output.jsonl') as reader:
    for obj_idx, test_sample in enumerate(reader):
        
        print(test_sample['scene_graph_set'])

In [ ]:
# Daniel Code
import jsonlines
with jsonlines.open('/mnt/storage_ssd/JOHAN_parallel_10_clip_output.jsonl') as reader:
    for obj_idx, test_sample in enumerate(reader):
        
        print(test_sample['scene_graph_set'])
        break